In [1]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./funlopt/funl_dynamics.jl")
include("./funlopt/funl_utils.jl")
include("./funlopt/funl_constraint.jl")
include("./trajopt/scaling.jl")
include("./funlopt/funl_synthesis.jl")
using BenchmarkTools
using SparseArrays

In [2]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/quadstar_N15_traj" my_dict
xnom = my_dict["x"]
unom = my_dict["u"]
tnom = my_dict["t"];
N = size(xnom,2) - 1
dtnom = zeros(N)
for i in 1:N
    dtnom[i] = tnom[i+1]-tnom[i]
end

In [3]:
dynamics = QuadrotorDynamics()
ix = dynamics.ix
iu = dynamics.iu
decay_rate = 0.1
DLMI = NonlinearDLMI(decay_rate,ix,iu,dynamics.Cv,dynamics.Dvu)
is = DLMI.is

1

In [4]:
iϕ = dynamics.iϕ
iv = dynamics.iv
@assert size(xnom,2) - 1 == N

In [5]:
idx = 4
A,B = diff(dynamics,xnom[:,idx],unom[:,idx])
Q = rand(ix,ix) .- 0.5

12×12 Matrix{Float64}:
  0.435005   -0.391634  -0.221851   …   0.13031     0.486538     0.487978
  0.348992   -0.399501   0.137431      -0.359935   -0.0993546   -0.01163
  0.0147516  -0.197506  -0.452445      -0.0939235   0.364636    -0.202046
 -0.438938   -0.213821   0.223148      -0.128466    0.1148      -0.102351
 -0.0142136   0.408653   0.160105      -0.124792   -0.432712    -0.114648
 -0.485187    0.382654   0.106738   …  -0.118845   -0.450843     0.296939
 -0.0508889   0.218385   0.314077       0.0632733  -0.451019    -0.315915
 -0.0547014   0.080082   0.420227      -0.471556   -0.00132702   0.443843
  0.446049    0.160744   0.447827       0.162817    0.465805    -0.0811101
 -0.239695    0.28847    0.243207       0.263515    0.00917905  -0.103668
 -0.143818    0.343588  -0.133957   …  -0.327484    0.355686    -0.476959
 -0.453979    0.447518   0.0738202      0.389458    0.0252176    0.0749097

In [6]:
Imat = sparse(Matrix(1.0I,ix,ix))

12×12 SparseMatrixCSC{Float64, Int64} with 12 stored entries:
 1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0

In [9]:
right = A + 0.5*DLMI.alpha*Imat
Aq = kron(Imat,right) + kron(right,Imat) * DLMI.Cn;

In [10]:
kron(right,Imat) - kron(right,Imat) * DLMI.Cn

144×144 Matrix{Float64}:
 0.0  0.0   0.0   -1.0   0.0   0.0   0.0   …  0.0   0.0       0.0        0.0
 0.0  0.05  0.0    0.0   0.0   0.0   0.0      0.0   0.0       0.0        0.0
 0.0  0.0   0.05   0.0   0.0   0.0   0.0      0.0   0.0       0.0        0.0
 0.0  0.0   0.0    0.05  0.0   0.0   0.0      0.0   0.0       0.0        0.0
 0.0  0.0   0.0    0.0   0.05  0.0   0.0      0.0   0.0       0.0        0.0
 0.0  0.0   0.0    0.0   0.0   0.05  0.0   …  0.0   0.0       0.0        0.0
 0.0  0.0   0.0    0.0   0.0   0.0   0.05     0.0   0.0       0.0        0.0
 0.0  0.0   0.0    0.0   0.0   0.0   0.0      0.0   0.0       0.0        0.0
 0.0  0.0   0.0    0.0   0.0   0.0   0.0      0.0   0.0       0.0        0.0
 0.0  0.0   0.0    0.0   0.0   0.0   0.0      0.0   0.0       0.0        0.0
 ⋮                             ⋮           ⋱  ⋮                          
 0.0  0.0   0.0    0.0   0.0   0.0   0.0   …  0.0   0.0       0.0        0.0
 0.0  0.0   0.0    0.0   0.0   0.0   0.0      0.0   0.

In [11]:
using BlockDiagonals

function create_block_diagonal(right::Matrix, n::Int)
    blocks = [right for _ in 1:n]
    return BlockDiagonal(blocks)
end
function optimized_kron_identity(right::Matrix{Float64})
    n = size(right, 1)
    m = size(right, 2)
    result = zeros(Float64, n*m, n*m)

    for i in 1:n
        row_index = (i-1)*n + 1
        col_index = (i-1)*m + 1
        result[row_index:row_index+n-1, col_index:col_index+m-1] = right
    end

    return result
end
function optimized_kron_identity_sparse(right::Matrix{T}) where T
    n = size(right, 1)
    m = size(right, 2)
    result = spzeros(T, n*m, n*m)

    for i in 1:n
        row_index = (i-1)*n + 1
        col_index = (i-1)*m + 1
        result[row_index:row_index+n-1, col_index:col_index+m-1] = right
    end

    return result
end

optimized_kron_identity_sparse (generic function with 1 method)

In [12]:
@benchmark kron(Imat,$right)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.729 μs …  1.042 ms  ┊ GC (min … max):  0.00% … 99.44%
 Time  (median):     2.292 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.039 μs ± 18.917 μs  ┊ GC (mean ± σ):  23.71% ±  4.50%

   █▁       ▄▆▆▂▂▁                                            
  ▅███▆▄▃▄▅███████▇▆▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  1.73 μs        Histogram: frequency by time        4.52 μs <

 Memory estimate: 9.95 KiB, allocs estimate: 9.

In [13]:
@benchmark optimized_kron_identity_sparse($right)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  15.542 μs …  2.629 ms  ┊ GC (min … max):  0.00% … 97.92%
 Time  (median):     19.584 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   24.380 μs ± 96.162 μs  ┊ GC (mean ± σ):  16.89% ±  4.25%

    ▆█▅▁ ▁▅▇▇▆▄▂▁▁▁                                            
  ▂▇████████████████▇▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  15.5 μs         Histogram: frequency by time          38 μs <

 Memory estimate: 87.34 KiB, allocs estimate: 105.

In [14]:
@benchmark optimized_kron_identity_sparse($right)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  15.750 μs …  68.739 ms  ┊ GC (min … max):  0.00% … 99.81%
 Time  (median):     19.167 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   32.620 μs ± 696.903 μs  ┊ GC (mean ± σ):  34.05% ±  4.27%

   ▃█▁ ▂▄▂                                                      
  ▃███████▇▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  15.8 μs         Histogram: frequency by time         51.9 μs <

 Memory estimate: 87.34 KiB, allocs estimate: 105.

In [15]:
@benchmark create_block_diagonal($right, ix)

BenchmarkTools.Trial: 10000 samples with 983 evaluations.
 Range (min … max):  58.452 ns …  13.770 μs  ┊ GC (min … max): 0.00% … 99.12%
 Time  (median):     59.257 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   67.911 ns ± 208.136 ns  ┊ GC (mean ± σ):  8.90% ±  5.08%

  █▇▄▂▁▂▃▆▅▃  ▁▁▂     ▁                                        ▂
  █████████████████▇▇██▇█▇▇██▇▇██▇▇▇▇▇▆▆▆▅▆▄▅▅▅▅▅▄▅▅▄▅▂▄▄▄▄▃▄▅ █
  58.5 ns       Histogram: log(frequency) by time      89.5 ns <

 Memory estimate: 144 bytes, allocs estimate: 1.

In [16]:
kron_  = kron(Imat,right)
_kron = kron(right,Imat);

In [17]:
Cn = sparse(DLMI.Cn);

In [18]:
@benchmark kron(right,Imat) * Cn

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):  5.188 μs … 623.271 μs  ┊ GC (min … max):  0.00% … 98.11%
 Time  (median):     6.493 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   7.802 μs ±  25.205 μs  ┊ GC (mean ± σ):  16.54% ±  5.08%

   ▆█▄      ▁▄▄▆▅▂                                             
  ▄███▅▄▃▃▄▆███████▇▆▅▄▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  5.19 μs         Histogram: frequency by time        11.4 μs <

 Memory estimate: 22.17 KiB, allocs estimate: 14.

In [24]:
@benchmark kron(right,Imat)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.139 μs … 380.176 μs  ┊ GC (min … max):  0.00% … 98.32%
 Time  (median):     2.736 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.421 μs ±  14.690 μs  ┊ GC (mean ± σ):  18.95% ±  4.39%

   █▁      ▃▆▄▇▄▄▄▁▁                                           
  ▇██▇▄▃▄▅██████████▇▆▅▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  2.14 μs         Histogram: frequency by time        5.03 μs <

 Memory estimate: 9.95 KiB, allocs estimate: 9.

In [19]:
@benchmark Cn * kron_

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.490 μs … 482.693 μs  ┊ GC (min … max):  0.00% … 98.31%
 Time  (median):     4.312 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.200 μs ±  18.104 μs  ┊ GC (mean ± σ):  15.70% ±  4.49%

  ▂▂▃     ▃▅▆▆▇█▅▂                                             
  ███▄▂▃▆██████████▇▆▆▄▄▃▃▃▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  3.49 μs         Histogram: frequency by time        7.49 μs <

 Memory estimate: 12.22 KiB, allocs estimate: 5.

In [20]:
@benchmark kron(Imat,Imat)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.038 μs … 149.363 μs  ┊ GC (min … max): 0.00% … 97.20%
 Time  (median):     1.079 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.269 μs ±   4.066 μs  ┊ GC (mean ± σ):  8.86% ±  2.75%

  ▃▆█▇▆▅▃▃▃▂▁▁    ▁▁▁ ▁▂▁▂▂▃▂▂▁▁▁▁  ▁                         ▂
  █████████████▇██████████████████▇██████▇██▇▇▇▆▇▇▆▅▆▇▄▅▅▅▅▅▅ █
  1.04 μs      Histogram: log(frequency) by time      1.74 μs <

 Memory estimate: 3.67 KiB, allocs estimate: 6.

In [21]:
@benchmark Matrix(1.0I,ix*ix,ix*ix)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):   1.806 μs …  1.782 ms  ┊ GC (min … max):  0.00% … 99.05%
 Time  (median):     14.704 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   21.871 μs ± 52.262 μs  ┊ GC (mean ± σ):  37.41% ± 16.80%

    █▄                                                         
  ▇▆██▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▂
  1.81 μs         Histogram: frequency by time         278 μs <

 Memory estimate: 162.05 KiB, allocs estimate: 2.

In [22]:
kron_ = kron(Q',B)
_kron = kron(B,Q')
Cm = sparse(DLMI.Cm);

In [23]:
sum(abs.(kron(B,Q')*Cm - Cn*kron_))

0.0

In [ ]:
@benchmark kron(B,Q')*Cm

In [ ]:
@benchmark Cn*kron_

In [33]:
using Random

In [111]:
A = rand(ix,ix) .- 0.5;
Q = rand(ix,ix)
Q = (Q+Q')
;

In [112]:
function evaluate(q)
    Q = reshape(q,(ix,ix))
    return vec(A * Q + Q' * A')
end

function eval_diff_numeric(q)
    ix = length(q)
    eps_x = Diagonal{Float64}(I, ix)
    fx = zeros(ix,ix)
    h = 2^(-18)
    for i in 1:ix
        fx[:,i] .= (evaluate(q+h*eps_x[:,i]) - evaluate(q-h*eps_x[:,i])) / (2*h)
    end
    return fx
end

eval_diff_numeric (generic function with 1 method)

In [113]:
fx_ = eval_diff_numeric(vec(Q));

In [114]:
Imat = sparse(Matrix(1.0I,ix,ix));

In [115]:
right = A
kron_ = create_block_diagonal(right,ix)
# fx = kron_ + Cn * kron_;
# fx = kron_ + kron(A,Imat)
# fx = kron_ + kron(A,Imat) * Cn
;

In [110]:
sum(abs.(fx-fx_))

4.854243185192786e-8

In [123]:


# Example usage
matrix = generate_squared_matrix(3)
println(matrix)

[1, 4, 5, 7, 8, 9]
